## A simple model for demand and supply of publicly-provided services in a city

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import gaussian_process
from matplotlib import pyplot as plt 
from enum import Enum
import os.path

In [ ]:
from cityItems import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [ ]:
gaussKern = gaussian_process.kernels.RBF
get_random_pos = lambda n: [tuple(
    np.round(np.random.uniform(low=[45.40,9.1],high=[45.50, 9.3], size=2),5)) for z in range(n)]

In [ ]:
get_random_pos(3)

In [ ]:
## ServiceUnit class
class ServiceUnit:
    def __init__(self, service, name='', position=(45.4641, 9.1919),
                 ageDiffusionIn={}, scaleIn=1, attributesIn={}):
        assert isinstance(position, tuple) & (len(position) == 2), 'Position must be a pair tuple' #will be replaced by nicer class
        assert isinstance(service, ServiceType), 'Service must belong to the Eum'
        assert isinstance(name, str), 'Name must be a string'
        
        assert (np.isscalar(scaleIn) & scaleIn>0) , 'Scale must be a positive scalar'
        assert isinstance(attributesIn, dict), 'Attributes can be provided in a dict'
        
        self.name = name
        self.service = service
        
        self.site = position  # a Service can have many sites, and a site is not uniquely assigned to a service
        
        self.scale = scaleIn # store scale info
        self.attributes = attributesIn# dictionary
        
        # how the service availablity area varies for different age groups
        if ageDiffusionIn==None:
            self.ageDiffusion = {g: (0.04 + .001*np.round(np.random.normal(),2))*scale for g in users} 
        else:
            assert set(ageDiffusionIn.keys()) <= set(AgeGroup.all()), 'Diffusion keys should be AgeGroups'
            #assert all
            self.ageDiffusion = ageDiffusionIn
        self.kernel = {g: gaussKern(length_scale=l) for g, l in self.ageDiffusion.items()}
        
        
        
    def evaluate(self, targetPositions, ageGroup):
        assert isinstance(targetPositions[0], tuple),'tuple list format expected for positions'
        reshapedPos = np.array(self.site).reshape(-1,2)
        # evaluate kernel to get level service score. If age group is not relevant to the service, return 0 as default
        if self.kernel.__contains__(ageGroup):
            score = self.kernel[ageGroup](targetPositions, reshapedPos)
            # check conversion from tuple to nparray
            #targetPositions= np.array(targetPositions)
            #score2 = self.kernel[ageGroup](targetPositions, reshapedPos)
            #assert all(score-score2==0)
        else:
            score = np.zeros_like(targetPositions)
        return np.squeeze(score)
    
    @property
    def users(self): return list(self.propagation.keys())

In [ ]:
zz ={a: 0 for a in AgeGroup.all()}

In [ ]:
#outScores = {(a, t): np.zeros(positions.shape[0]) for a in outputAgeGroups for t in outputServices}
#dimSpec = [('position', positions), ('agegroup', outputAgeGroups), ('servicetype', outputServices)]
#outXr = xr.DataArray(np.zeros([positions.shape[0], len(outputAgeGroups), len(outputServices)]),  dimSpec)

### Supply modelling
def evaluate_services_at(positions, unitsList, outputServices= [t for t in ServiceType]):
    # set all age groups as output default
    outputAgeGroups = AgeGroup.all()
    # initialise output
    outScores = {service: pd.DataFrame(np.zeros([len(positions), len(AgeGroup.all())]), 
                                 index=positions, columns=AgeGroup.all()) 
                 for service in outputServices}
    # loop over different services
    for thisServType in outputServices:
        serviceUnits = [u for u in unitsList if u.service == thisServType]
        if not serviceUnits:
            continue
        else:
            for thisAgeGroup in outputAgeGroups:
                unitValues = np.stack(list(map(lambda x: x.evaluate(positions, thisAgeGroup), serviceUnits)), axis=-1)
                # aggregate unit contributions according to the service type norm
                print(unitValues)
                outScores[thisServType][thisAgeGroup] = thisServType.aggregate_units(unitValues)
            
    return outScores
        

In [ ]:
test = [ServiceUnit(ServiceType.PoliceStation, 'Duomo'), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', position=(45.43, 9.201))]
evaluate_services_at(get_random_pos(50), test)


In [ ]:
class UnitFactory:
    def __init__(self, path):
        assert os.path.isfile(path), 'File "%s" not found' % path
        self.filepath = path
        self.rawData = []
        
    def load(self):
        self.rawData = pd.read_csv(self.filepath, sep=';', decimal=',')
        self.nUnits = self.rawData.shape[0]
        defaultLocationColumns = ['Lat', 'Long']
        if set(defaultLocationColumns).issubset(set(self.rawData.columns)):
            print('Location data found')
            locations = [tuple(self.rawData.loc[i, defaultLocationColumns]) for i in range(self.nUnits)]
            propertData = self.rawData.drop(defaultLocationColumns, axis=1)
        else:
            propertData = self.rawData
            locations = []
            
        return propertData, locations

    @staticmethod
    def createLoader(serviceType, path):
        if serviceType == ServiceType.School:
            return SchoolFactory(path)


class SchoolFactory(UnitFactory):
    
    def __init__(self, path):
        super().__init__(path)
        
    def load(self):
        (propertData, locations) = super().load()
        
        nameCol = 'DENOMINAZIONESCUOLA'
        typeCol = 'ORDINESCUOLA'
        
        typeAgeDict = {'SCUOLA PRIMARIA': {AgeGroup.ChildPrimary:1},
                      'SCUOLA SECONDARIA I GRADO': {AgeGroup.ChildMid:1},
                      'SCUOLA SECONDARIA II GRADO': {AgeGroup.ChildHigh:1},}
        schoolTypes = propertData[typeCol].unique()
        assert set(schoolTypes) <= set(typeAgeDict.keys()), 'Unrecognized types in input'
        
        scaleCol = propertData.ALUNNI
        #scaleData = 25*(scaleData/max(scaleData))
        
        unitList = []
                
        for scType in schoolTypes:
            bThisGroup = propertData[typeCol]==scType
            typeData = propertData[bThisGroup]
            typeLocations = [l for i,l in enumerate(locations) if bThisGroup[i]]
            # collect attendance data - can be scaled by this group but this affects city vs city comparisons
            scaleData = scaleCol[bThisGroup]
            
            for iUnit in range(typeData.shape[0]):
                rowData = typeData.iloc[iUnit,:]
                attrDict = {'level':scType}
                print(scaleData[iUnit])
                thisUnit = ServiceUnit(ServiceType.School, 
                        name=rowData[nameCol], 
                        position=typeLocations[iUnit], 
                        ageDiffusionIn=typeAgeDict[scType], 
                        scaleIn=scaleData[iUnit],
                        attributesIn=attrDict)
                
            unitList.append(thisUnit)
        
        return unitList

In [ ]:
## Load scuole!
scuoleFile =  'final/milano_datiScuole.csv'
oggetto = UnitFactory.createLoader(ServiceType.School, scuoleFile)

In [ ]:
## Plot tools
class unitPlotter:
    def __init__(self, unitList):
        assert isinstance(unitList, list), 'List expected'
        self.units = unitList
    
    def plot_locations(self):
        plt.figure()
        plt.scatter(self.latitude, self.longitude, s=self.scale)
        plt.axis('equal')
        plt.show()
    
    
    @property
    def longitude(self):
        return [unit.site[1] for unit in self.units]
    @property
    def latitude(self):
        return [unit.site[0] for unit in self.units]
    @property
    def scale(self):
        return [unit.scale for unit in self.units]

In [ ]:
tt = unitPlotter(oggetto.load())
tt.plot_locations()
#ax.set_aspect('equal', adjustable='box')

In [ ]:
### Demand modelling
class Household:
    def __init__(self, position=None, membersInput=None):
        # make defaults
        if not position: position = get_random_pos(1)
        if not membersInput: membersInput = {a: 1 for a in AgeGroup.all()}
        # expand input to all age group keys
        self.members = {a: membersInput.get(a, 0) for a in AgeGroup.all()}
        self.position = position
        self.export = pd.DataFrame(self.members, index=([self.position])) # precompute for speed

def evaluate_demand(householdList, outputServices= [t for t in ServiceType]):
    """ """
    # initialise output
    outDemand = dict()
    # consolidate positions. If two households share the same position, sum components.
    householdData = pd.concat([h.export for h in householdList])
    householdData['position'] = householdData.index 
    consolidated = householdData.groupby('position').sum()
    
    for thisServType in outputServices:
        outDemand[thisServType] = consolidated*thisServType.demandFactors
        
    return outDemand

In [ ]:
hhList =  [Household() for i in range(40)]
evaluate_demand(hhList)

In [ ]:
## Matching demand and supply
def get_satisfaction_indexes(householdList)